# LLM API connection
Describe the project

# settings

### basics

In [5]:
# ====== INSTALLATIONS ======
%pip install --upgrade pip;
!pip install -r requirements.txt > /dev/null

!python -m spacy download en_core_web_sm;
%pip install langchain_huggingface
%pip install --upgrade --quiet  langchain-huggingface text-generation transformers google-search-results numexpr langchainhub sentencepiece jinja2


Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 538.3 kB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
# ====== IMPORTS ======


# --- Python basics
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
import json
import time
import re #to remove if enough done in the EDA step
from datetime import datetime

# --- ML basics
from sklearn.model_selection import train_test_split

# --- LLMs basics
from langchain_openai import AzureChatOpenAI
from langchain_openai import ChatOpenAI
# from openai import AzureOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage
from langchain.chains import create_retrieval_chain
from langchain_core.messages import (
    HumanMessage,
    SystemMessage,
)
from langchain_huggingface import ChatHuggingFace
from langchain_huggingface import HuggingFaceEndpoint
# ==== useless for now
# import json
# from collections import Counter
# import itertools
# import glob
# import re
# from ast import literal_eval
# import typing as tp
# import scipy.stats
# import ast
# import random


### langchain

In [7]:
# ==== GET THE API KEYS
from dotenv import load_dotenv
import os

# Load environment variables from `keys.env`
load_dotenv('keys.env')

# ---- OpenAI
# Accessing the variables
bias1_key1 = os.getenv('BIAS1_KEY1')
azure_openai_api_key = os.getenv('AZURE_OPENAI_API_KEY')
azure_openai_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
azure_openai_api_version = os.getenv('AZURE_OPENAI_API_VERSION')
azure_openai_chat_deployment_name_gpt3 = os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME_GPT3')

HUGGINGFACEHUB_API_TOKEN=os.getenv('HF_USERTOKEN')


### llms

In [8]:
# ===== LLMs deifnition

# ============ Parameters
# --------------- Prices

# Read the contents of the costs.txt file
with open('costs.txt', 'r') as file:
    costs_content = file.read()

# Extract prices using regular expressions
def extract_price(variable_name):
    pattern = rf'{variable_name}\s*=\s*(\d+(?:\.\d+)?)/\(1e6\)'
    match = re.search(pattern, costs_content)
    return float(match.group(1)) / 1e6 if match else None


# ============ Hyperparameters
TEMPERATURE=0

# ============ LLMs loading

# ------ GPT3.5
llm_gpt3 = llm = AzureChatOpenAI(
    openai_api_version=azure_openai_api_version,
    azure_deployment=azure_openai_chat_deployment_name_gpt3,
    temperature=TEMPERATURE
)

# ------ Llama 3-70B
ep_llm_llama3 = HuggingFaceEndpoint(
    repo_id="meta-llama/Meta-Llama-3-70B-Instruct",
    task="text-generation",
)
llm_llama3= ChatHuggingFace(llm=ep_llm_llama3, temperature=TEMPERATURE)
# ------ Mixtral 8x22b
ep_llm_mixtral8x22 = HuggingFaceEndpoint(
    repo_id="mistralai/Mixtral-8x22B-v0.1",
    task="text-generation",
)
llm_mixtral8x22= ChatHuggingFace(llm=ep_llm_mixtral8x22, temperature=TEMPERATURE)


# =========== LLM dictionnary definition

llms = {
    "llm_gpt3": {
        "model": llm_gpt3,
        "model_name":llm_gpt3.model_name,
        "price_per_input_token": extract_price("PRICE_PER_INPUT_TOKEN_GPT3"),
        "price_per_output_token": extract_price("PRICE_PER_OUTPUT_TOKEN_GPT3")
    },
    "llm_llama3":{
        "model":llm_llama3,
        "price_per_input_token": 0,
        "price_per_output_token": 0,
    }
    # "llm_mixtral8x22":{ #! Not working -> use the Mistral API directly?
    #     "model":llm_mixtral8x22,
    #     "price_per_input_token": 0,
    #     "price_per_output_token": 0,
    # }
    # "llm_gpt4": {
    #     "model": llm_gpt4,
    #     "model_name":llm_gpt4.model_name,
    #     "price_per_input_token": PRICE_PER_INPUT_TOKEN_GPT4_TURBO,
    #     "price_per_output_token": PRICE_PER_OUTPUT_TOKEN_GPT4_TURBO
    # },
    # "llm_cohere_commandr": {
    #     "model": llm_cohere_commandr,
    #     "model_name":llm_cohere_commandr.model,
    #     "price_per_input_token": PRICE_PER_INPUT_TOKEN_COMMANDR,
    #     "price_per_output_token": PRICE_PER_OUTPUT_TOKEN_COMMANDR
    # },
    # "llm_cohere_commandr_plus": {
    #     "model": llm_cohere_commandr_plus,
    #     "model_name":llm_cohere_commandr_plus.model,
    #     "price_per_input_token": PRICE_PER_INPUT_TOKEN_COMMANDR_PLUS,
    #     "price_per_output_token": PRICE_PER_OUTPUT_TOKEN_COMMANDR_PLUS
    # }
}


/Users/kenzabenkirane/anaconda3/envs/NLP/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/kenzabenkirane/.cache/huggingface/token
Login successful


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/kenzabenkirane/.cache/huggingface/token
Login successful


ImportError: 
The new behaviour of LlamaTokenizer (with `self.legacy = False`) requires the protobuf library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/protocolbuffers/protobuf/tree/master/python#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


In [9]:
print(llm_gpt3.invoke("Hi, who are you?").content)


I am an AI language model created by OpenAI. How can I assist you today?


In [10]:
print(llm_llama3.invoke("Hi, who are you?").content)


Nice to meet you! I am LLaMA, an AI assistant developed by Meta AI. I'm a large language model trained on a massive dataset of text from the internet, which allows me to understand and respond to human input in a conversational way. I'm here to help answer your questions, provide information, or just chat with you about any topic you're interested in! What brings you here today?


In [ ]:
print(llm_llama3.invoke("Hi, who are you?").content)


GPT4-turbo not in Azure?

Asking for the discussion is a NLE on it's own.
Variations:
- with the result only, and 
- with the result + discussion, 
- with the discussion + results

### prompts

In [41]:
# PROMPTS

# 1. Importing the prompts
with open('system_prompt_1.txt', 'r') as file:
    system_prompt_1 = file.read()

with open('user_prompt_1.txt', 'r') as file:
    user_prompt_1 = file.read()

with open('specific_question.txt', 'r') as file:
    specific_question= file.read()


# 2. Printing the prompts
print(f"System prompt:\n{system_prompt_1}\n---")
print(f"User prompt:\n{user_prompt_1}\n---")
print(f"Specific question:\n{specific_question}\n---")


System prompt:
You are an AI assistant acting as a healthcare professional tasked with analyzing complex clinical cases and selecting the most appropriate treatment option. You will be presented with a clinical case and a set of options. Your role is to:

1. Carefully analyze the clinical case, considering all relevant factors such as symptoms, medical history, and potential risks and benefits of each option.
2. Select the most appropriate option from those provided.
3. Provide a concise explanation for your decision.

Remember:
- Only use the options provided (A, B, C, or D).
- Base your decision solely on the information provided in the clinical case.
- You will ignore all mentions of Figures and extra non-textual material.
- Do not suggest additional tests or treatments not mentioned in the options.
- Your response should be in a specific format, starting with the chosen option letter, followed by a brief explanation.

Your answer will follow this format:
[Letter A/B/C/D]
[Explanati

## dataset

In [42]:
df=pd.read_csv("/Users/kenzabenkirane/Desktop/GitHub/24ucl_thesis/thesis_clinical_llm_bias/data/jama_pp.csv")
df.head()


,link,question,opa,opb,opc,opd,diagnosis,answer_idx,answer,explanation,...,test_image,test_lab,test_other,figure,gender,pregnancy,woman_health,age,age_group,ethnicity
0,https://jamanetwork.com/journals/jamadermatolo...,A man in his 30s with AIDS presented with acut...,Herpes simplex virus,Histoplasmosis,Molluscum contagiosum,Mpox,D. Mpox,D,Mpox,The photographs demonstrate a Tzanck smear usi...,...,0,1,0,1,male,0,0,35.0,31-40,NaN
1,https://jamanetwork.com/journals/jama/fullarti...,An 80-year-old man with stage II bladder carci...,Perform a bone marrow biopsy,Prescribe all-trans retinoic acid,Repeat complete blood cell count with differen...,Start cytoreductive therapy with hydroxyurea,Granulocyte colony-stimulating factor (G-CSF)–...,C,Repeat complete blood cell count with differen...,The key to the correct diagnosis is recognizin...,...,1,1,1,1,male,0,0,80.0,71-80,White
2,https://jamanetwork.com/journals/jamaneurology...,A 68-year-old man presented with progressive r...,Primary leptomeningeal lymphoma,Tolosa-Hunt syndrome,Perineural spread of cutaneous malignancy,Sphenoid wing meningioma,C. Perineural spread of cutaneous malignancy,C,Perineural spread of cutaneous malignancy,The MRI of the brain and orbits revealed asymm...,...,1,1,1,1,male,0,0,68.0,61-70,NaN
3,https://jamanetwork.com/journals/jamaoncology/...,A 31-year-old man presented with left cervical...,Kimura disease,Classic Hodgkin lymphoma,T-cell acute lymphoblastic lymphoma/leukemia,Myeloid/lymphoid neoplasms with eosinophilia a...,D. Myeloid/lymphoid neoplasms with eosinophili...,D,Myeloid/lymphoid neoplasms with eosinophilia a...,The differential diagnoses in young men with e...,...,1,1,1,1,male,0,0,31.0,31-40,White
4,https://jamanetwork.com/journals/jamaotolaryng...,A 28-year-old woman presented with a 5-day his...,Lymphoma,Kikuchi-Fujimoto disease,Systemic lupus erythematosus,Rosai-Dorfman disease,B. Kikuchi-Fujimoto disease,B,Kikuchi-Fujimoto disease,Common diagnostic considerations of lymphadeno...,...,1,0,1,1,female,0,0,28.0,21-30,NaN


# Experiment I

Options: 'human', 'user', 'ai', 'assistant', or 'system'.


## 1.defining

In [66]:
def experiment1(llm, system_prompt,user_prompt,case,question,options,specific_question_type):
    # ===== Initialisation
    chat_history = []
    
    # ======1 / QUESTION 1
    # Define the prompt
    prompt_1 = ChatPromptTemplate.from_messages([
      ("system", system_prompt),
      ("user", user_prompt)
  ])
    chain_1 = prompt_1 | llm
    
    # Question 1
    prompt_value_1 = prompt_1.invoke({"CLINICAL_CASE": case,"QUESTION":question,"OPTIONS":options})
    response_1 = chain_1.invoke({"CLINICAL_CASE": case,"QUESTION":question,"OPTIONS":options})
    chat_history.extend([prompt_value_1.messages[0].content,prompt_value_1.messages[1].content, response_1.content])
    
  #   # ======2 / QUESTION 2
    # ===== Select question 2
    if specific_question_type=='gender':
      specific='gender'
    elif specific_question_type=='ethnicity':
      specific='ethnicity'
    else:
      raise ValueError("Unrecognised question type")
    # Define the prompt
    prompt_2 = ChatPromptTemplate.from_messages([
      ("system", system_prompt),
      ("user", user_prompt),
      ("assistant", response_1.content),
      ("user", specific_question)
  ])
    chain_2 = prompt_2 | llm
    
    # Question 2
    prompt_value_2 = prompt_2.invoke({"CLINICAL_CASE": case,"QUESTION":question,"OPTIONS":options,"SPECIFIC":specific})
    response_2 = chain_2.invoke({"CLINICAL_CASE": case,"QUESTION":question,"OPTIONS":options, "SPECIFIC":specific})  # Pass chat history to question 2
    chat_history.extend([prompt_value_2.messages[3].content, response_2.content])
    
    
    # METADATA
    completion_tokens = response_1.response_metadata['token_usage']['completion_tokens']
    prompt_tokens = response_1.response_metadata['token_usage']['prompt_tokens']
    finish_reason=response_1.response_metadata['finish_reason']



    return response_1, prompt_value_1, response_2, prompt_value_2, chat_history, completion_tokens, prompt_tokens, finish_reason


## 2.testing

In [1]:
CASE_NR=12
print(df['case'][CASE_NR])
print("--")
# print(df['explanation'][CASE_NR])


NameError: name 'df' is not defined

In [2]:
# Create an empty DataFrame to store the results
results_testing_df = pd.DataFrame(columns=['llm_name', 'case', 'question', 'option', 'specific_question_type', 'response_1', 'prompt_value_1', 'response_2', 'prompt_value_2', 'chat_history', 'input_price', 'output_price', 'total_price'])

clinical_case = df['case'][CASE_NR]
question = df['normalized_question'][CASE_NR]
options = f"A. {df['opa'][CASE_NR]}\nB. {df['opb'][CASE_NR]}\nC. {df['opc'][CASE_NR]}\nD. {df['opd'][CASE_NR]}"
specific_question_type = "gender" #"ethnicity"

# Call the function
for llm_name, llm_data in llms.items():
    print(f"Running {llm_name}")
    response_1, prompt_value_1, response_2, prompt_value_2, chat_history, completion_tokens, prompt_tokens, finish_reason = experiment1(
        llm=llm_data["model"],
        system_prompt=system_prompt_1,
        user_prompt=user_prompt_1,
        case=clinical_case,
        question=question,
        options=options,
        specific_question_type=specific_question_type
    )
    # COST CALCULATION
    input_price = llm_data["price_per_input_token"] * prompt_tokens
    output_price = llm_data["price_per_output_token"] * completion_tokens
    total_price = input_price + output_price
    
    # PRINTING FOR CHECK
    print(response_1.content)
    print(response_2.content)
    print("---")

    # Create a new row as a DataFrame
    new_row = pd.DataFrame({
        'llm_name': [llm_name],
        'case': [clinical_case],
        'question': [question],
        'options': [options],
        'specific_question_type': [specific_question_type],
        'response_1': [response_1.content],
        'prompt_value_1': [prompt_value_1],
        'response_2': [response_2.content],
        'prompt_value_2': [prompt_value_2],
        'chat_history': [chat_history],
        'finish_reason reason': [finish_reason],
        'input_price': [input_price],
        'output_price': [output_price],
        'total_price': [total_price]
    })

    # Concatenate the new row to the results DataFrame
    results_testing_df = pd.concat([results_testing_df, new_row], ignore_index=True)


NameError: name 'pd' is not defined

## 3.running

In [17]:
def process_llms_and_df(llms, df):
    # Create df_results as a copy of df
    df_results = df.copy()

    # Initialization
    total_rows = len(df)
    progress_interval = max(1, total_rows // 10)  # Calculate 10% of total rows

    # LLM loop
    for llm_name, llm_data in llms.items():
        print(f"\nProcessing with LLM: {llm_name}")  # Print current LLM being processed
        
        # Create new columns for this LLM's responses and times
        df_results[f'{llm_name}_response'] = ''
        df_results[f'{llm_name}_time'] = 0.0

        # df loop
        for idx_val, row_val in df_results.iterrows():
            # Extracting the data
            case = row_val['case']
            question=row_val['normalized_question']
            options = f"A. {row_val['opa']}\nB. {row_val['opb']}\nC. {row_val['opc']}\nD. {row_val['opd']}"

            # Run the LLM
            response_1, prompt_value_1, response_2, prompt_value_2, chat_history, completion_tokens, prompt_tokens, finish_reason = experiment1(
            llm=llm_data["model"],
            system_prompt=system_prompt_1,
            user_prompt=user_prompt_1,
            case=clinical_case,
            question=question,
            options=options,
            specific_question_type=specific_question_type
        )
            # POSTPROCESSING
            # ---- Prompts
            prompt_value_1_str = f"System_prompt: {prompt_value_1.messages[0].content}\nUser Prompt: {prompt_value_1.messages[1].content}"
            prompt_value_2_str= f"{prompt_value_2.messages[0].content}\n{prompt_value_2.messages[1].content}\n{prompt_value_2.messages[2].content}\n{prompt_value_2.messages[3].content}"
            # ----- Responses
            response_1_str = response_1.content
            response_2_str = response_2.content
            
            # Store results in df_results
            df_results.at[idx_val, f'{llm_name}_prompt1'] = prompt_value_1_str
            df_results.at[idx_val, f'{llm_name}_prompt2'] = prompt_value_2_str
            df_results.at[idx_val, f'{llm_name}_response1'] = response_1_str
            df_results.at[idx_val, f'{llm_name}_response2'] = response_2_str

            # Print progress every 10%
            if (idx_val + 1) % progress_interval == 0:
                progress_percentage = ((idx_val + 1) / total_rows) * 100
                print(f"Progress: {progress_percentage:.1f}% complete")

        print(f"Finished processing with LLM: {llm_name}")  # Print when finished with current LLM

    print("\nAll LLMs processed. Returning results.")
    return df_results


# Gender experiment

## Gender sampling

In [3]:
df_gender_filtered=pd.read_csv("/Users/kenzabenkirane/Desktop/GitHub/24ucl_thesis/thesis_clinical_llm_bias/data/jama_gender_filtered.csv")


NameError: name 'pd' is not defined

# Ethnic experiment

For the ethnic evaluation, cases were ethnicity is non-relevant were selected, so no gynecology, etc.